## Imports

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings
import math
import geopy.distance

from datetime import date, timedelta

In [7]:
# Convert cleaned CSV from Alteryx to parquet
(pd.read_csv('Merges/PETERSON_CLEANED.csv')
 .astype({'mark': str, 'file': str, 'dataset': str, 'comment': str})
 .to_parquet('Merges/PETERSON_CLEANED.parquet', engine='auto'))

C:\Users\ahernes\AppData\Local\Temp\1\ipykernel_14868\3160088194.py:2: DtypeWarning: Columns (18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  (pd.read_csv('Merges/PETERSON_CLEANED.csv')


## 1995-2012/MIDAS 1995-2000 Aggregation
Encompasses all datasets in 'MIDAS 1995-2000' directory parsing through '.001' files

In [2]:
# 232 '.001' files in MIDAS 1995-2000
err_list = []
master_list = []
file_list = []
midas_cols = ['date', 'time', 'lat', 'lon', 'depth_md', 'temp_air_md', 'wind_spd_md', 'wind_dir_md', 'barometer_md', 'bow_temp_md', 'salinity_md', 'sus_part_matter_md', 
        'photic_depth_md', 'chlor_md', 'neph_md', 'fluor_switch_md', 'event_mark_md', 'comment_num_md', '_drop', '_drop2']

def convert_midas_datetime(df):
    # Some entries contain 99-99-9999 for date and 99:99:99 for time
    df = df.rename(columns={'date_time': 'datetime'})
    df.datetime = pd.to_datetime(df.datetime, errors='coerce')
    df = df[pd.notnull(df.datetime)]
    return df

def convert_dms_decimal(value):
    tag = value[-1]
    direction = {'N':1, 'S':-1, 'E': 1, 'W':-1}

    second = int(value[-3:-1]) / 3600
    minute = int(value[-6:-4]) / 60
    degree = int(str(float(value[:-1]) / 100).split('.')[0])

    return (degree + minute + second) * direction[tag]


def convert_midas_latlon(df):
    df.lat = df.lat.apply(convert_dms_decimal)
    df.lon = df.lon.apply(convert_dms_decimal)

    # Filter out error values of 99 and 999 respectively
    df = df[abs(df.lat) < 99]
    df = df[abs(df.lon) < 999]
    return df


# Iterate over all files in 1995-2000
def read_midas_file():
    count = 0
    test_limit = math.inf
    for root, _, files in os.walk('Flow Through/1995-2012/MIDAS 1995-2000'):    
        for file in files:
            if file.endswith('.001') and file not in file_list:
                try:
                    print(f'Processing: {file}')
                    with warnings.catch_warnings(): # Catching a warning about parse_dates being retired
                        warnings.simplefilter("ignore")
                        file_list.append(file)
                        df = pd.read_csv(os.path.join(root, file), encoding='unicode_escape', names=midas_cols, header=None, parse_dates=[['date', 'time']]).drop(columns=['_drop', '_drop2'])
                        df['sus_part_matter_md'] = df['sus_part_matter_md'].astype(str)
                        df['photic_depth_md'] = df['photic_depth_md'].astype(str)
                        df = convert_midas_datetime(df)
                        df = convert_midas_latlon(df)
                        df['dataset'] = 'MIDAS'
                        df['file'] = file                   
                        master_list.append(df)

                    count += 1
                    if count == test_limit: return
                    
                except Exception as e:
                    err_list.append(file)
                    file_list.append(file)
                    print(f'ERROR: {e}')
                
read_midas_file()
df_midas_master = pd.concat(master_list, axis=0).reset_index(drop=True)
print(f'Datasets Aggregated: {len(master_list)}')
print(f'Errors: {len(err_list)}') 
print(err_list)

df_midas_master.to_csv('Merges/midas_test_merge.csv')
df_midas_master.to_parquet('Merges/midas_test_merge.parquet', engine='auto')       

Processing: 01040657.001
Processing: 01050741.001
Processing: 02290721.001
Processing: 03020914.001
Processing: 03290742.001
Processing: 03290809.001
Processing: 03291311.001
Processing: 03291341.001
Processing: 04040717.001
Processing: 04040750.001
Processing: 04040756.001
Processing: 04060804.001
Processing: 04060930.001
Processing: 05180748.001
Processing: 02080756.001
Processing: 02100733.001
Processing: 02250643.001
ERROR: file structure not yet supported
Processing: 03090617.001
ERROR: file structure not yet supported
Processing: 03160520.001
ERROR: Error tokenizing data. C error: Expected 38 fields in line 270, saw 40

Processing: 03170629.001
ERROR: Error tokenizing data. C error: Expected 38 fields in line 2005, saw 40

Processing: 11290520.001
Processing: 11290604.001
Processing: 11290633.001
Processing: 11291104.001
Processing: 11300658.001
Processing: 04040505.001
Processing: 04040628.001
Processing: 04041047.001
Processing: 04050621.001
Processing: 04070708.001
Processing:

C:\Users\ahernes\AppData\Local\Temp\1\ipykernel_14868\989357820.py:66: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_midas_master = pd.concat(master_list, axis=0).reset_index(drop=True)


Datasets Aggregated: 213
Errors: 4
['02250643.001', '03090617.001', '03160520.001', '03170629.001']


## 1995-2012/MOPED Aggregation

In [3]:
err_list = []
master_list = []
file_list = []
moped_cols = ['datetime', 'lat deg', 'lat min', 'lon deg', 'lon min', 'utm_north', 'utm_east', 'depth_mp', 'heading_mag_mp',
              'heading_gps_mp', 'speed_gps_mp', 'speed_water_mp', 'speed_wind_mp', 'wind_dir_mp', 'barometer_mp', 'temp_air_mp', 'spare_mp', 'chlor_mp',
              'turb_mp', 'temp_water_mp', 'salinity_mp', 'temp_bow_mp', 'mark_mp']

def convert_moped_datetime(df):
    df.datetime = pd.to_datetime(df.datetime, format='%m/%d/%Y %H:%M:%S', errors='coerce')
    df = df[pd.notnull(df.datetime)]
    return df

def convert_moped_latlon(df):
    # Just reassigning lat deg and lat min (col 1 and 2) rather than creating new columns and reordering
    df['lat deg'] = df['lat deg'].astype(int) + df['lat min'].astype(float) / 60
    df['lat min'] = df['lon deg'].astype(int) + df['lon min'].astype(float) / 60
    df = df.drop(columns=['lon deg', 'lon min']).rename(columns={'lat deg': 'lat', 'lat min': 'lon'})
    return df

def read_moped_file():
    count = 0
    test_limit = math.inf
    for root, _, files in os.walk('Flow Through/1995-2012/MOPED'):    
        for file in files:
            file = file.lower() # Some files end in '.TXT'
            if (file.endswith('.txt') or file.endswith('.csv')) and file[:5] not in file_list and "log" not in file:
            #if file == '05054_salinitiesforsteve.txt':
                try:
                    print(f'Processing: {file}')
                    file_start = file[:5]
                    file_list.append(file_start)
                    df = pd.read_csv(os.path.join(root, file), names=moped_cols, header=None).drop(columns=['utm_north', 'utm_east'])
                    df.mark_mp = df.mark_mp.astype(str)
                    df = convert_moped_datetime(df)
                    df = convert_moped_latlon(df)
                    df['dataset'] = 'MOPED'
                    df['file'] = file
                    master_list.append(df)

                    count += 1
                    if count == test_limit: return

                except Exception as e:
                    err_list.append(file)
                    file_list.append(file)
                    print(f'ERROR: {e}')

    
read_moped_file()
df_moped_master = pd.concat(master_list, axis=0).reset_index(drop=True)
print(f'Datasets Aggregated: {len(master_list)}')
print(f'Errors: {len(err_list)}')
print(err_list) 

df_moped_master.to_csv('Merges/moped_test_merge.csv')
df_moped_master.to_parquet('Merges/moped_test_merge.parquet', engine='auto')  

Processing: 00166.txt
Processing: 00193.txt
Processing: 00223.txt
Processing: 00249dat.txt
Processing: 00251dat.txt
Processing: 00284.txt
Processing: 00286.txt
Processing: 00312dat.txt
Processing: 00314dat.txt
Processing: 00347dat.txt
Processing: 00349dat.txt
Processing: 12193dat.csv
Processing: 12199dat.txt
Processing: 00221b.txt
Processing: 00195a.txt
Processing: 00165combo.txt
Processing: test165.txt
Processing: 00167.txt
Processing: 01037bdat.txt
Processing: 01039cdat.txt
Processing: 01046dat.txt
Processing: 01053dat.txt
Processing: 01057dat.txt
Processing: 01059dat.txt
Processing: 01066adat.txt
Processing: 01075dat.txt
Processing: 01086dat.txt
Processing: 01088dat.txt
Processing: 01100dat.txt
Processing: 01106dat.txt
Processing: 01114dat.txt
Processing: 01116dat.txt
Processing: 01144dat.txt
Processing: 01170dat.txt
Processing: 01172bdat.txt
Processing: 01198dat.txt
Processing: 01200dat.txt
Processing: 01254dat.txt
Processing: 01289dat.txt
Processing: 01291bdat.txt
Processing: 0133

## 1995-2012/MIDAS 1995-2000/94-99 ARCHFILE Aggregation

In [104]:
# Quick script to unzip all the subdirs containing the .DAT files first
import zipfile

arch_dir = 'Flow Through/1995-2012/MIDAS 1995-2000/94-99 ARCHFILE'
#os.chdir(arch_dir)

for root, dirs, files in os.walk(arch_dir):
    for file in files:
        file = file.lower()
        if file.endswith('.zip'):
            print(f'Unzipping: {file}')
            file_name = os.path.join(root, file) # get full path of files
            zip_ref = zipfile.ZipFile(file_name) # create zipfile object
            zip_ref.extractall(f'{arch_dir}/_extracted') # extract file to dir
            zip_ref.close() # close file
            #os.remove(file_name)

Unzipping: 94333.zip
Unzipping: 95018.zip
Unzipping: 95024.zip
Unzipping: 95038.zip
Unzipping: 95046.zip
Unzipping: 95053.zip
Unzipping: 95059.zip
Unzipping: 95066.zip
Unzipping: 95075.zip
Unzipping: 95082.zip
Unzipping: 95089.zip
Unzipping: 95094.zip
Unzipping: 95097.zip
Unzipping: 95101.zip
Unzipping: 95108.zip
Unzipping: 95109.zip
Unzipping: 95114.zip
Unzipping: 95117.zip
Unzipping: 95122.zip
Unzipping: 95129.zip
Unzipping: 95136.zip
Unzipping: 95164.zip
Unzipping: 95199.zip
Unzipping: 95228.zip
Unzipping: 95264.zip
Unzipping: 95296.zip
Unzipping: 95298.zip
Unzipping: 96016.zip
Unzipping: 96032.zip
Unzipping: 96037.zip
Unzipping: 96044.zip
Unzipping: 96052.zip
Unzipping: 96074.zip
Unzipping: 96086.zip
Unzipping: 96094.zip
Unzipping: 96109.zip
Unzipping: 96114.zip
Unzipping: 96122.zip
Unzipping: 96130.zip
Unzipping: 96164.zip
Unzipping: 96199.zip
Unzipping: 96226.zip
Unzipping: 96255.zip
Unzipping: 96290.zip
Unzipping: 96318.zip
Unzipping: 96352.zip
Unzipping: 97013.zip
Unzipping: 97

In [254]:
import utm
err_list = []
master_list = []
file_list = []
arch_cols = ['jdate', 'hour', 'utmn', 'utme', 'depth_ar', 'wind_spd_ar', 'wind_dir_ar', 'temp_ar', 'salinity_ar',
             'fvolt_ar', 'nvolt_ar', 'setting_ar', 'comment_ar', 'event_ar']

def convert_time_datetime(days, time):
    # Start as defined in .MTD file
    start = date(1899, 12, 31)
    datetime = str(start + timedelta(days))
    
    hours = time
    sec = hours * 3600
    m, s = divmod(sec, 60)
    h, m = divmod(m, 60)
    time = ("%d:%02d:%02d" % (h, m, s))

    return f'{datetime} {time}'

def convert_arch_datetime(df):
    df.jdate = df.jdate.astype(int)
    df.hour = df.hour.astype(float)
    df = df[df.jdate > 0]
    df.jdate = df.apply(lambda x: convert_time_datetime(x.jdate, x.hour), axis=1)
    df = df.rename(columns={'jdate': 'datetime'}).drop(columns=['hour'])
    df.datetime = pd.to_datetime(df.datetime)
    return df

def convert_utm_decimal(utme, utmn):
    # Bay Area is Zone 10 letter S according to https://upload.wikimedia.org/wikipedia/commons/b/b7/Universal_Transverse_Mercator_zones.svg
    return utm.to_latlon(utme, utmn, 10, 'S')

def convert_arch_latlon(df):
    df.utme = df.utme.astype(float)
    df.utmn = df.utmn.astype(float)

    # Read .MTD file to understand utme/utmn conversion from km to m
    df['temp'] = df.apply(lambda x: convert_utm_decimal((x.utme * 1000), ((x.utmn * 1000) + 4000000)), axis=1)
    df = df.rename(columns={'utmn': 'lat', 'utme': 'lon'})
    df[['lat', 'lon']] = df.temp.apply(pd.Series)
    df = df.drop(columns=['temp'])
    return df 

def read_arch_file():
    count = 0
    test_limit = math.inf
    for root, _, files in os.walk('Flow Through/1995-2012/MIDAS 1995-2000/94-99 ARCHFILE/_extracted'):
        for file in files:
            if file.endswith('.DAT') and file not in file_list:
                try:
                    print(f'Processing: {file}')
                    file_list.append(file)
                    df = pd.read_csv(os.path.join(root, file), names=arch_cols, header=0)
                    # Cleaning invalid rows from metric (not comment) columns
                    df = df.replace('NO DATA', np.nan).replace('NODATA', np.nan).dropna(subset=arch_cols[:-3])
                    df.comment_ar = df.comment_ar.astype(str)
                    df.depth_ar = df.depth_ar.astype(float)
                    df.temp_ar = df.temp_ar.astype(float)
                    df.salinity_ar = df.salinity_ar.astype(float)
                    df.fvolt_ar = df.fvolt_ar.astype(float)
                    df.nvolt_ar = df.nvolt_ar.astype(float)
                    df = convert_arch_latlon(df)
                    df = convert_arch_datetime(df)
                    df['dataset'] = 'ARCHFILE'
                    df['file'] = file
                    master_list.append(df)

                    count += 1
                    if count == test_limit: return df

                except Exception as e:
                    err_list.append(file)
                    file_list.append(file)
                    print(f'ERROR: {e}')

read_arch_file()
df_arch_master = pd.concat(master_list, axis=0).reset_index(drop=True)
print(f'Datasets Aggregated: {len(master_list)}')
print(f'Errors: {len(err_list)}') 

df_arch_master.to_csv('Merges/arch_test_merge.csv')
df_arch_master.to_parquet('Merges/arch_test_merge.parquet', engine='auto')

Processing: 94333.DAT
Processing: 95018.DAT
Processing: 95024.DAT
Processing: 95038.DAT
Processing: 95046.DAT
Processing: 95053.DAT
Processing: 95059.DAT
Processing: 95066.DAT
Processing: 95075.DAT
Processing: 95082.DAT
Processing: 95089.DAT
Processing: 95094.DAT
Processing: 95097.DAT
Processing: 95101.DAT
Processing: 95108.DAT
Processing: 95109.DAT
Processing: 95114.DAT
Processing: 95117.DAT
Processing: 95122.DAT
Processing: 95129.DAT
Processing: 95136.DAT
Processing: 95164.DAT
Processing: 95199.DAT
Processing: 95228.DAT
Processing: 95264.DAT
Processing: 95296.DAT
Processing: 95298.DAT
Processing: 96016.DAT
Processing: 96032.DAT
Processing: 96037.DAT


C:\Users\ahernes\AppData\Local\Temp\1\ipykernel_19124\2867345825.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.jdate = df.apply(lambda x: convert_time_datetime(x.jdate, x.hour), axis=1)


Processing: 96044.DAT
Processing: 96052.DAT
Processing: 96074.DAT
Processing: 96086.DAT
Processing: 96094.DAT
Processing: 96109.DAT
Processing: 96114.DAT
Processing: 96122.DAT
Processing: 96130.DAT
Processing: 96164.DAT
Processing: 96199.DAT
Processing: 96226.DAT
Processing: 96255.DAT
Processing: 96290.DAT
Processing: 96318.DAT
Processing: 96352.DAT
Processing: 97013.DAT
Processing: 97028.DAT
Processing: 97045.DAT
Processing: 97050.DAT
Processing: 97057.DAT
Processing: 97065.DAT
Processing: 97070.DAT
Processing: 97076.DAT
Processing: 97091.DAT
Processing: 97100.DAT
Processing: 97112.DAT
Processing: 97120.DAT
Processing: 97134.DAT
Processing: 97161.DAT
Processing: 97196.DAT
Processing: 97217.DAT
Processing: 97252.DAT
Processing: 97280.DAT
Processing: 97310.DAT
Processing: 97343.DAT
Processing: 98006.DAT
Processing: 98020.DAT
Processing: 98042.DAT
Processing: 98049.DAT
Processing: 98050.DAT
Processing: 98057.DAT
Processing: 98064.DAT
Processing: 98071.DAT
Processing: 98076.DAT
Processing

## 2013-2023 Aggregation (Most Recent)

In [4]:
err_list = []
master_list = []
file_list = []

def convert_rec_datetime(df):
    df = df.rename(columns={'Date': 'datetime'})
    df.datetime = pd.to_datetime(df.datetime, format='%m/%d/%Y %H:%M:%S', errors='coerce')
    df = df[pd.notnull(df.datetime)]
    return df

def convert_rec_latlon(df):
    # Just reassigning lat deg and lat min (col 1 and 2) rather than creating new columns and reordering
    df['Lat deg'] = df['Lat deg'].astype(int) + df['Lat min'].astype(float) / 60
    df['Lat min'] = df['Lon deg'].astype(int) + df['Lon min'].astype(float) / 60
    df = df.drop(columns=['Lon deg', 'Lon min']).rename(columns={'Lat deg': 'lat', 'Lat min': 'lon'})
    return df

def read_rec_file():
    count = 0
    test_limit = math.inf
    for root, _, files in os.walk('Flow Through/Recent'):
        for file in files:
            file = file.lower()
            if file.endswith('dat.txt') and file not in file_list and "log" not in file:
                try:
                    print(f'Processing: {file}')
                    file_list.append(file)
                    df = pd.read_csv(os.path.join(root, file))
                    df = convert_rec_datetime(df)
                    df = convert_rec_latlon(df)
                    df['dataset'] = 'REC'
                    df['file'] = file
                    master_list.append(df)

                    count += 1
                    if count == test_limit: return
                except Exception as e:
                    err_list.append(file)
                    file_list.append(file)
                    print(f'ERROR: {e}')

            # 2015 has no dat.txt files, only .xlsx
            if file.endswith('.xlsx') and os.path.exists(f'Flow Through/Recent/2015/{file}'):
                try:
                    print(f'Processing: {file}')
                    file_list.append(file)
                    df = pd.read_excel(os.path.join(root, file)).drop(columns=['UTM North', 'UTM East'])
                    df = convert_rec_datetime(df)
                    df = convert_rec_latlon(df)
                    df['dataset'] = 'REC'
                    df['file'] = file
                    master_list.append(df)

                    count += 1
                    if count == test_limit: return

                except Exception as e:
                    err_list.append(file)
                    file_list.append(file)
                    print(f'ERROR: {e}')
                


read_rec_file()
df_rec_master = pd.concat(master_list, axis=0).reset_index(drop=True).drop(columns=['UTM North', 'UTM East'])
df_rec_master['dataset'] = '2013-2023'

df_rec_master.Chlor = df_rec_master.Chlor.combine_first(df_rec_master['Chlor (volts)'])
df_rec_master.Chlor = df_rec_master.Chlor.combine_first(df_rec_master['Chloro (volts)'])
df_rec_master.Turb = df_rec_master.Turb.combine_first(df_rec_master['Turb (volts)'])
#df_rec_master['Air Temp'] = df_rec_master['Air Temp'].combine_first(df_rec_master.Temp)

print(f'Datasets Aggregated: {len(master_list)}')
print(f'Errors: {len(err_list)}')
print(err_list) 

df_rec_master.to_csv('Merges/rec_test_merge.csv')
df_rec_master.to_parquet('Merges/rec_test_merge.parquet', engine='auto')

Processing: 13015dat.txt
Processing: 13057dat.txt
Processing: 13071dat.txt
Processing: 13078dat.txt
Processing: 13085dat.txt
Processing: 13108dat.txt
Processing: 13113dat.txt
Processing: 13137dat.txt
Processing: 13196dat.txt
Processing: 13204dat.txt
Processing: 13240dat.txt
Processing: 13269dat.txt
Processing: 13297dat.txt
Processing: 13298dat.txt
Processing: 13323dat.txt
Processing: 13329dat.txt
Processing: 13337dat.txt
Processing: 14014dat.txt
Processing: 14024dat.txt
Processing: 14031dat.txt
Processing: 14042dat.txt
Processing: 14055dat.txt
Processing: 14070dat.txt
ERROR: 'DataFrame' object has no attribute 'datetime'
Processing: 14084dat.txt
Processing: 14098dat.txt
Processing: 14105dat.txt
Processing: 14113dat.txt
Processing: 14127dat.txt
Processing: 14133dat.txt
Processing: 14157dat.txt
Processing: 14161dat.txt
Processing: 14189adat.txt
Processing: 14189dat.txt
Processing: 14196dat.txt
Processing: 14216dat.txt
Processing: 14224dat.txt
Processing: 14246dat.txt
Processing: 14259dat

## Master Aggregation
Aggregate 4 subdatasets

In [5]:
#df_arch = pd.read_parquet('Merges/arch_test_merge.parquet')
df_midas = pd.read_parquet('Merges/midas_test_merge.parquet')
df_moped = pd.read_parquet('Merges/moped_test_merge.parquet')
df_rec = pd.read_parquet('Merges/rec_test_merge.parquet')

length = len(df_midas) + len(df_moped) + len(df_rec)
order = ['datetime', 'lat', 'lon', 'depth', 'photic_depth', 'wind_spd', 'wind_dir', 'water_temp', 'air_temp', 'bow_temp',
         'barometer', 'chlor', 'turbidity', 'suspended_part', 'salinity', 'fluor_switch', 'comment', 'mark', 'dataset', 'file']

# prep df_arch
#df_arch = df_arch.rename(columns={'depth_ar': 'depth',  'wind_spd_ar': 'wind_spd', 'wind_dir_ar': 'wind_dir', 
#                                  'temp_ar': 'water_temp', 'salinity_ar': 'salinity', 'fvolt_ar' : 'chlor', 'nvolt_ar': 'turbidity',
#                                  'setting_ar': 'fluor_switch', 'comment_ar': 'comment', 'event_ar': 'mark'})

#print(df_arch.head())

# prep df_midas
df_midas = df_midas.rename(columns={'depth_md': 'depth', 'temp_air_md': 'air_temp', 'wind_spd_md': 'wind_spd',
                                    'wind_dir_md': 'wind_dir', 'barometer_md': 'barometer', 'bow_temp_md': 'bow_temp',
                                    'salinity_md': 'salinity', 'sus_part_matter_md': 'suspended_part',
                                    'photic_depth_md': 'photic_depth', 'chlor_md': 'chlor', 'neph_md': 'turbidity',
                                    'fluor_switch_md': 'fluor_switch', 'comment_num_md': 'comment', 'event_mark_md': 'mark'})

#print(df_midas.head())

# prep df_moped
df_moped = df_moped.drop(columns=['heading_mag_mp', 'heading_gps_mp', 'speed_gps_mp', 'speed_water_mp', 'spare_mp'])
df_moped = df_moped.rename(columns={'depth_mp': 'depth', 'speed_wind_mp': 'wind_spd', 'wind_dir_mp': 'wind_dir',
                                    'temp_water_mp': 'water_temp', 'temp_air_mp': 'air_temp', 'barometer_mp': 'barometer', 'chlor_mp': 'chlor',
                                    'turb_mp': 'turbidity', 'salinity_mp': 'salinity', 'temp_bow_mp': 'bow_temp', 'mark_mp': 'mark'})

#print(df_moped.head())


# prep df_rec
df_rec = df_rec.drop(columns=['Head mag', 'Head Gps', 'Spd Gps', 'Spd Wat', 'Spare', 'Chlor (volts)', 'Turb (volts)',
                              'Chloro (volts)', 'Scufa Fluor', 'Temp', 'PF Fo', 'PF Fm', 'PF blank', 'PF Fv', 'PF yield'])
df_rec = df_rec.rename(columns={'Depth': 'depth', 'Spd Wind': 'wind_spd', 'Wind dir': 'wind_dir', 'Air Temp': 'air_temp',
                                'Water Temp': 'water_temp', 'Sal': 'salinity', 'Chlor': 'chlor', 'Turb': 'turbidity',
                                'Mark': 'mark', 'Bow Temp': 'bow_temp', 'Baro': 'barometer'})

#print(df_rec.head())

df_master = (pd.concat([df_midas, df_moped, df_rec], ignore_index=True)
            .reset_index(drop=True)
            .reindex(columns=order)
            .sort_values(by='datetime'))

df_master.comment = df_master.comment.astype(str)
df_master.mark = df_master.mark.astype(str)

assert(length == len(df_master))
#print(df_master.columns)
#print(df_master)

df_master.to_parquet('Merges/PETERSON_MASTER.parquet', engine='auto')
df_master.to_csv('Merges/PETERSON_MASTER.csv')

## Peterson Master Dataset Metadata

In [6]:
df = pd.read_parquet('Merges/PETERSON_MASTER.parquet')
print(f'PQ file size: {round((os.path.getsize("Merges/PETERSON_MASTER.parquet") / (1024*1024)), 2)}mb')
print(f'Total Samples: {len(df)}')
print(f'Earliest Sample {df.datetime.min()}')
print(f'Latest Sample: {df.datetime.max()}')
print(f'Datetime Dupes: \n{df.groupby("datetime").datetime.value_counts().sort_values(ascending=False)}')

PQ file size: 90.67mb
Total Samples: 2495711
Earliest Sample 1988-01-09 11:11:46
Latest Sample: 2023-04-27 16:27:57
Datetime Dupes: 
datetime
2021-12-06 11:55:32    13
2021-12-06 11:55:36    10
2021-12-06 11:55:12    10
2021-12-06 11:55:53     9
2021-12-06 11:55:24     8
                       ..
2003-05-01 11:58:03     1
2003-05-01 11:58:08     1
2003-05-01 11:58:13     1
2003-05-01 11:58:18     1
2023-04-27 16:27:57     1
Name: count, Length: 2489659, dtype: int64
